In [59]:

# Only need to run this one time to get python-docx and openpyxl installed in the Env
# !pip install python-docx
# !pip install pandas openpyxl

In [22]:
## Lego superpowers engaged
from docx import Document
import pandas as pd
import os
import re

In [36]:
## Identify a table with just links so we can throw it away
def is_links_table(table):
    # Check if the table is a 1x1 table and contains URLs
    if len(table.rows) == 1 and len(table.rows[0].cells) == 1:
        cell_text = table.rows[0].cells[0].text
        # Check for URL patterns or additional information text
        if re.search(r'https?://', cell_text) or re.search(r'additional information', cell_text):
            return True
    return False

## Extract all of the tables into dataFrames - return a list of dataFrames
def extract_tables_from_docx(docx_path):
    doc = Document(docx_path)
    tables_list = []
    for table in doc.tables:
        if is_links_table(table):
            continue  # Skip this table
        
        data = [[cell.text for cell in row.cells] for row in table.rows]
        df_table = pd.DataFrame(data[1:], columns=data[0])  # Use the first row as column headers
        tables_list.append(df_table)
    return tables_list

def process_folder(folder_path):
    all_tables = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".docx"):
            docx_path = os.path.join(folder_path, filename)
            tables = extract_tables_from_docx(docx_path)
            # Add a column to each table indicating the filename
            for table in tables:
                table['Source File'] = filename  # Add filename as a column
            all_tables.extend(tables)
    return all_tables


In [37]:
# ## Define where our first document lives
# docx_path = r'unusnldwp2sampledata\LIBERIA.docx'

## Define the folder where all documents live
folder_path = r'unusnldwp2sampledata'

In [55]:
# ## Test extraction with just one file
# tables = extract_tables_from_docx(docx_path)
# tables[0].head()

In [40]:
all_tables = process_folder(folder_path)

In [57]:
all_tables[1].head()

,Country,Explanatory notes,Example: LIBERIA,,,,,Source File
0,Main event,Modulating event that produces one or more haz...,"Heavy Rainfall, SLR/Coastal Flooding in Montse...",,,,,LIBERIA.docx
1,Hazard,Single hazardous event causing loss and damage,Floods resulting from heavy Rainfall in margib...,,,,,LIBERIA.docx
2,Description & context,Describe the event and its context,"In recent years, climate change related coasta...",,,,,LIBERIA.docx
3,,,,,Needs,Sources of support,Gaps,LIBERIA.docx
4,1. Anticipatory Arrangement (Pre-Phase 1: befo...,Anticipatory arrangements are those that trigg...,Flood mapping has been done to identify hotspo...,,,,,LIBERIA.docx


In [66]:
output_folder_path = 'unusnldwp2sampledata/processedData'

# Loop through each DataFrame in the list
for dataframe in all_tables:
    original_filename = dataframe['Source File'].iloc[0]  # Access the first item for the filename
    # Remove any file extension from the original_filename
    original_filename_without_extension = os.path.splitext(original_filename)[0]
    # Construct the new filename by adding '_tables' suffix and the .xlsx extension
    new_filename = f"{original_filename_without_extension}_tables.xlsx"
    # Create the full path to the new file
    full_path = os.path.join(output_folder_path, new_filename)
    
    # Save the dataframe to an Excel file in the specified folder, excluding the index
    dataframe.to_excel(full_path, index=False)  # Add `columns=[col for col in dataframe.columns if col != 'originalFileName']` to exclude the filename column

print("All DataFrames have been saved to Excel files in the 'processeddata' folder.")


All DataFrames have been saved to Excel files in the 'processeddata' folder.


list